# COSC522 Assignment 3 - Motion Classifier

### Team 3: Purnachandra Anirudh Gajjala, Gabriel Abeyie, Cameron Adkins

In [1]:
# Numpy.
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

# Need plots.
import matplotlib.pyplot as plt

# Pandas.
import pandas as pd

# Machine learning toolkit.
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import *
from sklearn.preprocessing import Normalizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score

# Scipy for fft's and the like.
import scipy as sc
import scipy.io.wavfile as wavfile
from scipy import signal
from scipy.fftpack import fft, fftfreq
from scipy import stats

# Seaborn for plots.
import seaborn as sns

# Ipython for basic visual output types.
import IPython

# Standard Python libs.
import os
import glob
import csv

from dataclasses import dataclass

In [2]:
# Classes def.

@dataclass
class CartesianSeries:
    t: list;
    x: list;
    y: list;
    z: list;

@dataclass
class QuaternionSeries:
    t:     list;
    qx:    list;
    qy:    list;
    qz:    list;
    qw:    list;
    pitch: list;
    yaw:   list;
    roll:  list;


class MotionSample:
    def __init__(self, directory, label):
        self.dir   = directory;
        self.label = label;
        
        # We'll assume data is from Sensor Logger.
        orient_file = directory + "/Orientation.csv";
        gyro_file   = directory + "/Gyroscope.csv";
        accel_file  = directory + "/Accelerometer.csv";
        
        uncal_gyro_file  = directory + "/GyroscopeUncalibrated.csv";
        uncal_accel_file = directory + "/AccelerometerUncalibrated.csv";
        
        #total_accel_file = directory + "/TotalAcceleration.csv";
        
        # Load everything.
        self.gyro   = self.__loadxyz(gyro_file);
        self.accel  = self.__loadxyz(accel_file);
        self.orient = self.__loadpyr(orient_file);

        self.uncal_gyro  = self.__loadxyz(uncal_gyro_file);
        self.uncal_accel = self.__loadxyz(uncal_accel_file);
        
        #self.total_accel = self.__loadxyz(total_accel_file);
        
        # Run preprocessing.
        self.__preprocess();

    def __loadxyz(self, filename):
        df = pd.read_csv(filename);
        
        ret = CartesianSeries(0, 0, 0, 0);
        
        ret.t = df["seconds_elapsed"];
        ret.x = df["x"];
        ret.y = df["y"];
        ret.z = df["z"];
        
        return ret;
        
    def __loadpyr(self, filename):
        df = pd.read_csv(filename);
    
        ret = QuaternionSeries(0, 0, 0, 0, 0, 0, 0, 0);
    
        ret.t = df["seconds_elapsed"];
        ret.qx = df["qx"];
        ret.qy = df["qy"];
        ret.qz = df["qz"];
        ret.qw = df["qw"];
        
        ret.pitch = df["pitch"];
        ret.yaw   = df["yaw"];
        ret.roll  = df["roll"];
        
        return ret;
        
    def plots(self):
        print("This data is sourced from: " + self.dir);
        self.__plotxyz(self.gyro,        "Gyroscope Capture - " + self.label);
        self.__plotxyz(self.accel,       "Accelerometer Capture - " + self.label);
        self.__plotxyz(self.uncal_gyro,  "Uncalibrated Gyroscope Capture - " + self.label);
        self.__plotxyz(self.uncal_accel, "Uncalibrated Accelerometer Capture - " + self.label);
        self.__plotpyr(self.orient,      "Orientation Capture - " + self.label);
        
    def __plotxyz(self, cart_series, title):
        #plt.figure(figsize = (12, 8));
        plt.figure();
        
        plt.plot(cart_series.t, cart_series.x, label = "x(t)");
        plt.plot(cart_series.t, cart_series.y, label = "y(t)");
        plt.plot(cart_series.t, cart_series.z, label = "z(t)");

        plt.title(title);
        plt.xlabel("t");
        plt.legend();
        plt.show();
    
    def __plotpyr(self, quat_series, title, use_quat = False):
        #plt.figure(figsize = (12, 8));
        plt.figure();
        
        if (use_quat):
            plt.plot(quat_series.t, quat_series.qx, label = "qx(t)");
            plt.plot(quat_series.t, quat_series.qy, label = "qy(t)");
            plt.plot(quat_series.t, quat_series.qz, label = "qz(t)");
            plt.plot(quat_series.t, quat_series.qw, label = "qw(t)");
        else:
            plt.plot(quat_series.t, quat_series.pitch, label = "pitch(t)");
            plt.plot(quat_series.t, quat_series.yaw,   label = "yaw(t)");
            plt.plot(quat_series.t, quat_series.roll,  label = "roll(t)");  

        plt.title(title);
        plt.xlabel("t");
        plt.legend();
        plt.show();
                
    def features(self):
        foo = 0;
        # Windowing of the samples.
        # Feature selection.
        #   - FFT should provide some nice frequency bins to work off of.
        #   - Amplitude could be a nice way to detect stairs.
        #   - Signal/Noise ratio again? 
        
    def __get_windows(self):
        foo = 0;
        # Windowing should be aware of discontinuities.
        # There's something like this in Lab 2.
        
    def __fft(self, n_point = 1024):
        foo = 0;
        # Multi-axis fft, copy from Lab 2 with a couple tweaks.
        if (self.fft_data == None or n_point != self.fft_data[2]):
            freq = fftfreq(n = n_point, d = (1 / self.fs))[:int(n_point / 2)];
            fft_y = fft(self.x, n_point)[:int(n_point / 2)];
            mag = np.abs(fft_y);

            self.fft_data = (freq, mag, n_point);

        return self.fft_data;
        
    def __chop_ends(self):
        foo = 0;
        # Should remove starts/stops where rough data lies.
        
        
    def __resample(filename):
        foo = 0;
        # Resample the data and get a more consistant fs.
        # SciPy can do this: https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.resample.html
        # Might be better than pandas method.
        # Should also be aware of discontinuities.
        df = pd.read_csv(filename)
        df['seconds_elapsed'] = pd.to_datetime(df["seconds_elapsed"], unit='s')
        df = df.resample('10ms', on="seconds_elapsed").mean()
        df = df.interpolate()
        df = df.reset_index(level=0)
        d = datetime(1970, 1, 1, 0, 0, 0, 0)
        df["seconds_elapsed"] = (df["seconds_elapsed"]-d).dt.total_seconds()

  return df
        
    def __apply_lpf(self):
        foo = 0;
        # Can probably copy this over from Lab 2, but needs to apply to all axis.
        num, denom = signal.butter(lpf_order, cutoff_freq, fs = self.fs, btype = 'lowpass', analog = False);
        self.x = signal.lfilter(num, denom, self.x);
        self.y = signal.lfilter(num, denom, self.y);
        self.z = signal.lfilter(num, denom, self.z);

        # Reset cached data.
        self.fft_data     = None;
        self.spectro_data = None;

        return self.x, self.y, self.z;
        
    def __preprocess(self):
        foo = 0;
        # Call all our preprocessing here (resample, chop_ends, lpf, etc)
        #self.apply_norm();
        self.apply_lpf(15000);




In [3]:
# Load everything.
PWD = os.getcwd();
TRAINING_SAMPLES = PWD + "/training_samples";

def load_samples(samples_dir):
    class_dirs = glob.glob(samples_dir + "/*")

    samples = [];

    for class_dir in class_dirs:
        label = os.path.basename(class_dir);
        class_dir_glob = glob.glob(class_dir + "/*");

        print("Reading files for label '" + label + "'");

        for sample_dir in class_dir_glob:
            print(len(samples), ":", sample_dir);
            sample = MotionSample(sample_dir, label);
            samples.append(sample);
        
    return samples;

samples = load_samples(TRAINING_SAMPLES);

Reading files for label 'walk'
0 : /Users/gpanirudh/Downloads/Academics/UTK/Semester-1/ML/cosc522_lab3-master/training_samples/walk/walking_14
1 : /Users/gpanirudh/Downloads/Academics/UTK/Semester-1/ML/cosc522_lab3-master/training_samples/walk/walking_13
2 : /Users/gpanirudh/Downloads/Academics/UTK/Semester-1/ML/cosc522_lab3-master/training_samples/walk/walking_25
3 : /Users/gpanirudh/Downloads/Academics/UTK/Semester-1/ML/cosc522_lab3-master/training_samples/walk/walking_22
4 : /Users/gpanirudh/Downloads/Academics/UTK/Semester-1/ML/cosc522_lab3-master/training_samples/walk/walking_23
5 : /Users/gpanirudh/Downloads/Academics/UTK/Semester-1/ML/cosc522_lab3-master/training_samples/walk/walking_24
6 : /Users/gpanirudh/Downloads/Academics/UTK/Semester-1/ML/cosc522_lab3-master/training_samples/walk/walking_12
7 : /Users/gpanirudh/Downloads/Academics/UTK/Semester-1/ML/cosc522_lab3-master/training_samples/walk/walking_15
8 : /Users/gpanirudh/Downloads/Academics/UTK/Semester-1/ML/cosc522_lab3-m

In [4]:
%matplotlib notebook
samples[87].plots()

This data is sourced from: /Users/gpanirudh/Downloads/Academics/UTK/Semester-1/ML/cosc522_lab3-master/training_samples/stairs_up/climbingupstairs_19


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
# Try the model.
fv = [];
labels = [];

for sample in samples:
    fv.append(sample.features());
    labels.append(sample.label);

# Scale.
scaler = RobustScaler();
fv = scaler.fit_transform(fv);
    
# Split the data.
x_train, x_test, y_train, y_test = train_test_split(fv, labels, test_size = 0.30, random_state = 64);

# Train.
dt = RandomForestClassifier();
dt.fit(x_train, y_train);

# Testing the model.
cv_scores = cross_val_score(dt, x_train, y_train, cv = 10);

print('Average Cross Validation Score from Training:', cv_scores.mean(), sep = '\n', end = '\n\n\n');

y_pred = dt.predict(x_test);
cm = confusion_matrix(y_test, y_pred);
cr = classification_report(y_test, y_pred);

print('Confusion Matrix:', cm, sep = '\n', end = '\n\n\n');
print('Missing classifications (if any):', set(y_test) - set(y_pred));
print('Test Statistics:', cr, sep = '\n', end = '\n\n\n');
print('Testing Accuracy:', accuracy_score(y_test, y_pred));

ValueError: Expected 2D array, got 1D array instead:
array=[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.